In [32]:
import re

with open("inputs/Day_16.txt") as f:
    puzzle_data = f.read()
    
def part_1_solution(raw_data):
    rules, nearby_tickets = parse_input(raw_data)
    
    allowed_range = set()
    for rule_range in rules.values():
        allowed_range |= rule_range
    
    error_rate = 0
    
    for ticket in nearby_tickets:
        for value in ticket:
            if value not in allowed_range:
                error_rate += value
    
    return error_rate

def parse_input(raw_input):
    raw_rules, raw_my_ticket, raw_nearby_tickets = raw_input.split("\n\n")
    
    rules = parse_rules(raw_rules)
    raw_nearby_tickets = raw_nearby_tickets.split("\n")[1:]
    nearby_tickets = list()
    for raw_ticket in raw_nearby_tickets:
        ticket = [int(field) for field in raw_ticket.split(",")]
        nearby_tickets.append(ticket)
    
    return rules, nearby_tickets

RULE_RE = re.compile(r'(?P<name>.*): (?P<range_1_start>\d+)-(?P<range_1_end>\d+) or (?P<range_2_start>\d+)-(?P<range_2_end>\d+)')

def parse_rules(raw_rules):
    rules = dict()
    
    for raw_rule in raw_rules.split('\n'):
#         print(raw_rule)
        match = RULE_RE.match(raw_rule)
        assert(match)
        allowed_range = set()
        allowed_range |= set(range(int(match['range_1_start']), int(match['range_1_end']) + 1))
        allowed_range |= set(range(int(match['range_2_start']), int(match['range_2_end']) + 1))
        
        rules[match['name']] = allowed_range
    
    return rules

In [33]:
from helpers import test_single_case

test_input = """\
class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12\
"""
test_single_case(part_1_solution, 71, test_input)

PASSED (in 0.06 [ms])


In [34]:
%%time
print(f"Part 1 solution: {part_1_solution(puzzle_data)}")

Part 1 solution: 23122
CPU times: user 3.08 ms, sys: 0 ns, total: 3.08 ms
Wall time: 2.95 ms


In [84]:
import re
from collections import defaultdict

with open("inputs/Day_16.txt") as f:
    puzzle_data = f.read()
    
def part_2_solution(raw_data):
    rules, my_ticket, nearby_tickets = parse_input(raw_data)
    
    valid_tickets = get_valid_tickets(rules, nearby_tickets)
    valid_tickets.append(my_ticket)
    
    matching_columns = get_possible_matchings(rules, valid_tickets)
    
    # get final ordering of the columns
    idx_to_fields = dict()
    while matching_columns:
        # get column with len of matching idxs == 1
        assigned_idxs = set()
        columns_to_delete = list()
        for name, matching_idxs in matching_columns.items():
            if len(matching_idxs) == 1:
                found_idx = matching_idxs.pop()
                assigned_idxs.add(found_idx)
                idx_to_fields[found_idx] = name
                columns_to_delete.append(name)
                
        # delete matched columns
        for column in columns_to_delete:
            del matching_columns[column]
            
        # update remaining columns idxs
        for name, matching_idxs in matching_columns.items():
            matching_idxs.difference_update(assigned_idxs)
           
        
    # get final answer
    ans = 1
    
    for column_idx, value in enumerate(my_ticket):
        if idx_to_fields[column_idx].startswith("departure"):
            ans *= value
            
    return ans

def get_possible_matchings(rules, tickets):
    remaining_fields = list(rules.keys())
    matching_columns = defaultdict(set)
    
    
    while remaining_fields:
        current_field = remaining_fields.pop()
        for column_idx in range(len(tickets[0])):
            match = True
            for ticket in tickets:
                if ticket[column_idx] not in rules[current_field]:
                    match = False
                    break
            
            if match:
                matching_columns[current_field].add(column_idx)
                
    return matching_columns
    
def get_valid_tickets(rules, tickets):
    allowed_range = set()
    for rule_range in rules.values():
        allowed_range |= rule_range
    
    valid_tickets = list()
    
    for ticket in tickets:
        valid = True
        for value in ticket:
            if value not in allowed_range:
                valid = False
                break
        
        if valid:
            valid_tickets.append(ticket)
    
    return valid_tickets

def parse_input(raw_input):
    raw_rules, raw_my_ticket, raw_nearby_tickets = raw_input.split("\n\n")
    
    rules = parse_rules(raw_rules)
    
    raw_my_ticket = raw_my_ticket.split("\n")[-1]
    my_ticket = [int(field) for field in raw_my_ticket.split(",")]
    
    raw_nearby_tickets = raw_nearby_tickets.split("\n")[1:]
    nearby_tickets = list()
    for raw_ticket in raw_nearby_tickets:
        ticket = [int(field) for field in raw_ticket.split(",")]
        nearby_tickets.append(ticket)
    
    return rules, my_ticket, nearby_tickets

RULE_RE = re.compile(r'(?P<name>.*): (?P<range_1_start>\d+)-(?P<range_1_end>\d+) or (?P<range_2_start>\d+)-(?P<range_2_end>\d+)')

def parse_rules(raw_rules):
    rules = dict()
    
    for raw_rule in raw_rules.split('\n'):
#         print(raw_rule)
        match = RULE_RE.match(raw_rule)
        assert(match)
        allowed_range = set()
        allowed_range |= set(range(int(match['range_1_start']), int(match['range_1_end']) + 1))
        allowed_range |= set(range(int(match['range_2_start']), int(match['range_2_end']) + 1))
        
        rules[match['name']] = allowed_range
    
    return rules

In [87]:
%%time
print(f"Part 2 solution: {part_2_solution(puzzle_data)}")

Part 2 solution: 362974212989
CPU times: user 9.07 ms, sys: 0 ns, total: 9.07 ms
Wall time: 8.59 ms
